## Pretrained DenseNet201 model

In [2]:
# 3 channel image
# no bounding box + pretrained data
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np

def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        img_idx = i
        temp_images.append([])
        temp_images[i] = cv2.resize(train_images[img_idx],(64,64))
        temp_images[i] = cv2.cvtColor(temp_images[i],cv2.COLOR_GRAY2BGR)
        
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('train/train_images.pkl')
    labels = pd.read_csv('train/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],0),(labels[:train_size], labels[train_size:train_size+valid_size],0)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)


(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 0)

X_train = get_data(X_train)
X_test = get_data(X_test)

y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 64, 64

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 64, 64, 3)
X_test = X_test.reshape(X_test.shape[0], 64, 64, 3)


X_train/=255
X_test/=255
X_train.shape

number_of_classes = 10


Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

# Build DenseNet201 model

from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception

from keras.applications.densenet import DenseNet201
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet_v2 import MobileNetV2

model = DenseNet201(include_top=True, weights=None, input_tensor=None, input_shape=(64,64,3), pooling="avg", classes=10)


model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
gen.fit(X_train)


train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = gen.flow(X_test, Y_test, batch_size=64)

#check point
from keras.callbacks import ModelCheckpoint
filepath="model_densenet/weights-improvement-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='auto', period=1)
callbacks_list = [checkpoint]

#run model
history_MobileNetV2_64 = model.fit_generator(train_generator, steps_per_epoch=38000//64, epochs=100,validation_data=test_generator, validation_steps=2000//64,callbacks=callbacks_list)


score = model.evaluate(gen.standardize(X_test), Y_test)
print()
print('Test accuracy: ', score[1])





X_train original shape (38000, 64, 64, 3)
y_train original shape (38000,)
X_test original shape (2000, 64, 64, 3)
y_test original shape (2000,)
Epoch 1/100
593/593 [==============================] - 130s 220ms/step - loss: 1.0269 - acc: 0.6604 - val_loss: 3.2126 - val_acc: 0.4204

Epoch 00001: saving model to model_densenet/weights-improvement-01-0.4204.hdf5
Epoch 2/100
593/593 [==============================] - 96s 163ms/step - loss: 0.4112 - acc: 0.8728 - val_loss: 1.9331 - val_acc: 0.6653

Epoch 00002: saving model to model_densenet/weights-improvement-02-0.6653.hdf5
Epoch 3/100
593/593 [==============================] - 93s 156ms/step - loss: 0.3117 - acc: 0.9033 - val_loss: 0.6925 - val_acc: 0.8161

Epoch 00003: saving model to model_densenet/weights-improvement-03-0.8161.hdf5
Epoch 4/100
593/593 [==============================] - 94s 159ms/step - loss: 0.2713 - acc: 0.9171 - val_loss: 2.0108 - val_acc: 0.6229

Epoch 00004: saving model to model_densenet/weights-improvement-04-0.6

Epoch 76/100
593/593 [==============================] - 90s 152ms/step - loss: 0.0253 - acc: 0.9916 - val_loss: 0.1789 - val_acc: 0.9561

Epoch 00076: saving model to model_densenet/weights-improvement-76-0.9561.hdf5
Epoch 77/100
593/593 [==============================] - 90s 152ms/step - loss: 0.0236 - acc: 0.9921 - val_loss: 0.1477 - val_acc: 0.9613

Epoch 00077: saving model to model_densenet/weights-improvement-77-0.9613.hdf5
Epoch 78/100
593/593 [==============================] - 90s 152ms/step - loss: 0.0250 - acc: 0.9920 - val_loss: 0.1546 - val_acc: 0.9654

Epoch 00078: saving model to model_densenet/weights-improvement-78-0.9654.hdf5
Epoch 79/100
593/593 [==============================] - 90s 152ms/step - loss: 0.0250 - acc: 0.9923 - val_loss: 0.2088 - val_acc: 0.9478

Epoch 00079: saving model to model_densenet/weights-improvement-79-0.9478.hdf5
Epoch 80/100
593/593 [==============================] - 90s 152ms/step - loss: 0.0195 - acc: 0.9939 - val_loss: 0.2096 - val_acc: 0.